<font size="20">Final Map</font>

In [ ]:
# The import command allows us to call on various libraries
import folium
import numpy as np
import pandas as pd
import scipy.stats as stats
import geopandas as gpd
import matplotlib.pyplot as plt
import branca.colormap as cm
from geopy.geocoders import MapBox
from IPython.display import clear_output

# This allows us to create interactive plots with matplotlib
%matplotlib notebook

# You need this to query mapbox
api_key=""

if api_key == "":
    print('Please enter your API key to continue!')
else:
    print('Packages Loaded')

# Importing Census Data


In [ ]:
# the .read_file() function reads shapefiles
Toronto_CT = gpd.read_file('Data/CensusTracts/SimplyAnalytics_Shapefiles_2021-08-05_05_16_56_b1eb67b672fbcfbefd2c809a81d68228.shp')
print(Toronto_CT.crs)
Toronto_CT = Toronto_CT.rename(columns={
'VALUE0': 'Aboriginal identity, 2016',
'VALUE1': 'Population, 2016',
'VALUE2': 'Total visible minority population, 2016'
                    })

Toronto_CT['NonWhitePCT'] = Toronto_CT[['Aboriginal identity, 2016',
'Total visible minority population, 2016']].sum(axis=1)/Toronto_CT['Population, 2016']*100

# .to_crs()changes the coordinate system
# Provincial_Data = Provincial_Data.to_crs('EPSG:4326')
# .to_file() saves our data to the specified format
print('Data Converted')
print(Toronto_CT.NonWhitePCT.describe())
Toronto_CT.to_file("Data/Toronto_CT.json", driver = "GeoJSON")
Toronto_CT.head()

# Importing Police Involved Deaths Data


In [ ]:
PID = pd.read_csv('Data/Police_Inovolved_Deaths_Canada.csv',index_col='id_victim',parse_dates=['date'])
# print(PID.columns)
PID['postal_code'] = PID['postal_code'].fillna('Unspecified')
PID['postal_code'] = PID['postal_code'].fillna('Unspecified')
PID_BC = PID.loc[((PID['prov']=='BC')&(PID['postal_code']!='Unspecified'))].copy()
print(PID_BC['postal_code'].count())
# print(PID_BC['city_town'].unique())
PID_BC['city_town'] = PID_BC['city_town'].str.split(';').str[0]
# PID_BC['city_town']
# We'll create empty columns for latitude & longitude
# We'll create a "notes" column as well
PID_BC['address_intersection']=PID_BC['address_intersection'].fillna('')
PID_BC['race']=PID_BC['race'].fillna('Unspecified')
PID_BC['gender']=PID_BC['gender'].fillna('Unspecified')
PID_BC['latitude'] = np.nan
PID_BC['longitude'] = np.nan
PID_BC['geocoding_Notes'] = ''
print(PID_BC.columns)

# Geocoding

In [ ]:
geolocator = MapBox(api_key=api_key)

i = 0
for index, row in PID_BC.iterrows():
    attempt = ' '.join([row['address_intersection'], row['city_town'], row['prov'], row['postal_code'],'Canada'])
    try:
        # Progress report
        if i % 25 == 0:
            clear_output()
            print("PROGRESS: {} out of {}. Currently at: {}".format(i, len(PID_BC), i))

        g = geolocator.geocode(attempt,timeout=3)
    #     print(i,g.latitude)
        PID_BC.loc[PID_BC.index==index,
                    ['latitude','longitude']]=g.latitude,g.longitude
    except:
        
        Police_Killings.loc[Police_Killings['postal_code']==row['postal_code'],
                    'geocoding_Notes']='Geocoding Failed'
        pass
    i += 1
print('Geocoding Done')


In [ ]:
Failures = (PID_BC.loc[PID_BC['geocoding_Notes']=='Geocoding Failed',
                          ['prov','postal_code']])
print(Failures)


In [ ]:
# "def" defines our function "plot_point()", which takes five "arguments":
    # Map: The map you're working with
    # X & Y: lattitude & longitude
    # Popup_Text: What do we want the popup to say?
    # Color: We'll set a defualt, but we can override with what colour do we want
        # Everything else will remain the same for every point, so we can set them as default values
    # Defaults can be over written by assigning them anoter value
def plot_point(Map,X,Y,Popup_Text,Color='olive',Radius=5,Opacity=.75,LineColor='black',LineWidth=.15):
    folium.CircleMarker(
        # The coordiatnates
        location=[X,Y],
        # Text description
        popup=Popup_Text,
        # sets the fill color for the point
        fill_color=Color,
        # Size of the marker
        radius=Radius,
        # Opacity of the circle
        fill_opacity = Opacity,
        # Sets the line color for the edge
        color=LineColor,
        # Width of the border line
        line_weight=LineWidth,
    ).add_to(Map)

# Make a webmap showing all incidents in BC & census data for metro vancouver

In [ ]:

BC_Map = folium.Map(
    location=['49.260438','-123.088896'],
    zoom_start=11,
    tiles='Stamen Toner'
)


# The .features.Chloropleth() function allows us to classify our data
# We give function the location of our GeoJSON file

folium.features.Choropleth('Data/MVan_CT.json',
                           # It will match the geometry data up with a pandas or geopandas dataframe
                            data=MVan_CT,
                            columns=['spatial_id','NonWhitePCT'],
                           # They key in the GeoJSON file to match by
                            key_on='feature.properties.spatial_id',
                           # If we define bins, it will split where we tell it to
                            bins = [0,25,50,75,101],
                            fill_color='PuRd',
                            fill_opacity = 1,
                            smooth=2,
                           # The layer name
                            name='Non-White Population %',
                           # The legend label
                            legend_name='Non-White',
                           # Whether it shows up by default or must be turned on with the legend
                            show=False
                          ).add_to(BC_Map)


colormap = cm.linear.PuRd_05.scale(MVan_CT['Population, 2016'].min(), MVan_CT['Population, 2016'].max())
colormap = cm.LinearColormap(['white','blue'],vmin= MVan_CT['Population, 2016'].min(),vmax=MVan_CT['Population, 2016'].max())
# colormap = colormap.to_step(index=[MVan_CT['Population, 2016'].min(),
#                                    MVan_CT['Population, 2016'].quantile(.25),
#                                    MVan_CT['Population, 2016'].quantile(.5), 
#                                    MVan_CT['Population, 2016'].quantile(.75), 
#                                    MVan_CT['Population, 2016'].max()],
#                            round_method =None)
colormap.caption = 'Total Population %'
# colormap.caption = 'Police Killings/ Year / Million People (2000-2017)'
colormap.add_to(BC_Map)

folium.GeoJson(
    'Data/MVan_CT.json',
    name='Total Population',
    smooth_factor=1.75,
    style_function = lambda x:{'color':'black',
                               "weight": 1,
                               "fillOpacity": 1,
                              'fillColor':colormap(x['properties']['Population, 2016'])
                              },
    tooltip=folium.features.GeoJsonTooltip(fields=['Aboriginal identity, 2016',
    'Total visible minority population, 2016',
    'NonWhitePCT'],
       aliases=['Indigineous Population',
                'Visible Minorit (non-Indigenou) Poulation','NonWhitePCT']
                                          ),
    show = True
).add_to(BC_Map)



group = folium.FeatureGroup(name='Deaths')
for index, row in PID_BC.iterrows():
    # if the geocoding didn't fail, we'll plot the point, colored by the province
    if row['geocoding_Notes'] != 'Geocoding Failed':
#         print(row['race'], row['gender'],str(row['age']))
        plot_point(Map=group,
                   X=row['latitude'],
                   Y=row['longitude'],
                   Popup_Text=row['race']+' '+ row['gender'] + ' '+ str(row['age'])\
                     + ' '+ row['city_town'] + ' '+ row['prov']+ ' '+ row['postal_code'],
                   Color='red'#color_Scheme[row['prov']]
                  )

group.add_to(BC_Map)

folium.LayerControl().add_to(BC_Map)
BC_Map

In [ ]:
BC_Map.save('PoliceViolenceIncidents_BC.html')
print('Map Saved!')